In [1]:
from dotenv import load_dotenv
load_dotenv()
from import_neo.base import BaseNeoImporter
from import_neo.base_api import API2NeoImporter

## Make sure to run Local Neo4j

In [2]:
RUN_SPACES=False

# Use Backup in Digital Ocean Spaces

In [3]:
if RUN_SPACES:
    services_path = 'results/services.json.gzip'
    questions_path = 'results/questions.json.gzip'
    
    impo_services = BaseNeoImporter()
    impo_services.run(services_path)
    print('Services Data \n')
    print(impo_services.data[0])
    
    
    impo_q = BaseNeoImporter(node_type='Questions')
    impo_q.run(questions_path)
    impo_q.data[0]
    print('Questions Data \n')
    print(impo_q.data[0])

## Use API to Import to Neo4j

In [9]:
if not RUN_SPACES:
    api_path = 'http://localhost'
    q = API2NeoImporter(node_type='Questions', api_path=api_path)
    q.run('questions')
    print(q.data[0])
    
    s = API2NeoImporter(node_type='Services', api_path=api_path)
    s.run('services')
    print(s.data[0])

{'id': 1, 'question': 'Is anyone scaring, threatening or hurting you or your children?', 'tags': ['Domestic Violence', 'Shelter', 'Family'], 'main_tag': 'Family', 'mongo_id': 1, 'name': 'Is anyone scaring, threatening or hurting you or your children?'}
{'name': 'S. AZ Gender Alliance', 'phone': 5204777096, 'address': '2030 EAST BROADWAY', 'general_topic': 'LGBTQ', 'tags': ['LGBTQ'], 'city': 'Tuscon', 'state': 'AZ', 'lat': 32.2210873, 'lon': -110.94121, 'zip_code': 85719, 'web_site': 'http://www.sagatucson.org/', 'days': None, 'hours': None, 'id': '6374335957ba17e9ea1a2d85', 'mongo_id': '6374335957ba17e9ea1a2d85', 'main_tag': 'LGBTQ'}


* https://neo4j.com/docs/graph-data-science/current/algorithms/node-similarity/
* https://neo4j.com/docs/graph-data-science/current/algorithms/knn/
* https://neo4j.com/docs/graph-data-science/current/algorithms/bfs/
* https://neo4j.com/docs/graph-data-science/current/machine-learning/linkprediction-pipelines/link-prediction/